In [1]:
import os
import math
import glob
import numpy as np 
from astropy.io import fits


def open_fits(filename):
    fitsFile = fits.open(filename)
    fitsData = fitsFile[0].data
    fitsHdr = fitsFile[0].header

    return fitsData, fitsHdr


def write_fits(filepath, filename, data, hdr):
    try:
        fits.writeto(filepath+filename, data, hdr, overwrite=True)
        return 0
    except:
        return 1
    

def single_image(filepath, filename, darkFilename=None):
    r = 0
    rawData, rawHdr = open_fits(filepath+filename)
    
    if darkFilename is None:
        prcData = rawData
    else:
        darkData, darkHdr = open_fits(darkFilename)
        prcData = np.subtract(rawData, darkData)

    tmpFilename = filename.split('raw')[1]
    prcFilename = 'dsi'+tmpFilename
    r_ = write_fits(filepath, prcFilename, prcData, rawHdr)
    if r_ != 0:
        r = r_
        
    return r


def loop_thru_dir(filepath, darkFilename=None):
    r = 0
    directoryList = glob.glob(filepath+'raw-*')
    directoryList.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    
    for filename in directoryList:
        filename_ = filename.replace(filepath, '')
        r_ = single_image(filepath, filename_, darkFilename)
        if r_ != 0:
            r = r_
            
    return r
        

In [4]:
filePath = '/Users/aidancgray/Documents/SCORPIO/SVC/Images/08-11-2023/good/'
#fileName = 'raw-00000001.fits'
darkFile = '/Users/aidancgray/Documents/SCORPIO/SVC/Images/08-11-2023/darks.fits'

In [ ]:
single_image(filePath, fileName, darkFile)

In [5]:
loop_thru_dir(filePath, darkFile)

0